In [3]:
import numpy as np
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM

clusters_txt = '''
piśmiennicze: pisak flamaster ołówek długopis pióro
małe_ssaki: mysz szczur chomik łasica kuna bóbr
okręty: niszczyciel lotniskowiec trałowiec krążownik pancernik fregata korweta
lekarze: lekarz pediatra ginekolog kardiolog internista geriatra
zupy: rosół żurek barszcz
uczucia: miłość przyjaźń nienawiść gniew smutek radość strach
działy_matematyki: algebra analiza topologia logika geometria
budynki_sakralne: kościół bazylika kaplica katedra świątynia synagoga zbór
stopień_wojskowy: chorąży podporucznik porucznik kapitan major pułkownik generał podpułkownik
grzyby_jadalne: pieczarka borowik gąska kurka boczniak kania
prądy_filozoficzne: empiryzm stoicyzm racjonalizm egzystencjalizm marksizm romantyzm
religie: chrześcijaństwo buddyzm islam prawosławie protestantyzm kalwinizm luteranizm judaizm
dzieła_muzyczne: sonata synfonia koncert preludium fuga suita
cyfry: jedynka dwójka trójka czwórka piątka szóstka siódemka ósemka dziewiątka
owady: ważka biedronka żuk mrówka mucha osa pszczoła chrząszcz
broń_biała: miecz topór sztylet nóż siekiera
broń_palna: karabin pistolet rewolwer fuzja strzelba
komputery: komputer laptop kalkulator notebook
kolory: biel żółć czerwień błękit zieleń brąz czerń
duchowny: wikary biskup ksiądz proboszcz rabin pop arcybiskup kardynał pastor
ryby: karp śledź łosoś dorsz okoń sandacz szczupak płotka
napoje_mleczne: jogurt kefir maślanka
czynności_sportowe: bieganie skakanie pływanie maszerowanie marsz trucht
ubranie:  garnitur smoking frak żakiet marynarka koszula bluzka sweter sweterek sukienka kamizelka spódnica spodnie
mebel: krzesło fotel kanapa łóżko wersalka sofa stół stolik ława
przestępca: morderca zabójca gwałciciel złodziej bandyta kieszonkowiec łajdak łobuz
mięso_wędliny wieprzowina wołowina baranina cielęcina boczek baleron kiełbasa szynka schab karkówka dziczyzna
drzewo: dąb klon wiąz jesion świerk sosna modrzew platan buk cis jawor jarzębina akacja
źródło_światła: lampa latarka lampka żyrandol żarówka reflektor latarnia lampka
organ: wątroba płuco serce trzustka żołądek nerka macica jajowód nasieniowód prostata śledziona
oddziały: kompania pluton batalion brygada armia dywizja pułk
napój_alkoholowy: piwo wino wódka dżin nalewka bimber wiśniówka cydr koniak wiśniówka
kot_drapieżny: puma pantera lampart tygrys lew ryś żbik gepard jaguar
metal: żelazo złoto srebro miedź nikiel cyna cynk potas platyna chrom glin aluminium
samolot: samolot odrzutowiec awionetka bombowiec myśliwiec samolocik helikopter śmigłowiec
owoc: jabłko gruszka śliwka brzoskwinia cytryna pomarańcza grejpfrut porzeczka nektaryna
pościel: poduszka prześcieradło kołdra kołderka poduszeczka pierzyna koc kocyk pled
agd: lodówka kuchenka pralka zmywarka mikser sokowirówka piec piecyk piekarnik
'''

words_to_embed = set()
for line in clusters_txt.split('\n'):
    parts = line.split()
    words_to_embed.update(parts[1:])

model_name_p = "flax-community/papuGaPT2"
tokenizer_p = AutoTokenizer.from_pretrained(model_name_p)
model_p = AutoModelForCausalLM.from_pretrained(model_name_p).cpu()
embeddings_weight = model_p.transformer.wte.weight.detach().numpy()

p_dict = {}
for word in words_to_embed:
    input_text = " " + word
    token_ids = tokenizer_p.encode(input_text)
    vectors = embeddings_weight[token_ids]
    final_vector = np.mean(vectors, axis=0)
    p_dict[word] = final_vector / np.linalg.norm(final_vector)

model_name_bert = "allegro/herbert-base-cased"
tokenizer_bert = AutoTokenizer.from_pretrained(model_name_bert)
model_bert = AutoModel.from_pretrained(model_name_bert).cpu()

bert_dict = {}
for word in words_to_embed:
    token_ids = tokenizer_bert(word, return_tensors='pt')['input_ids']
    output = model_bert(input_ids=token_ids)
    vectors = output.last_hidden_state.detach().numpy()[0, 1:-1, :]
    final_vector = np.mean(vectors, axis=0)
    bert_dict[word] = final_vector / np.linalg.norm(final_vector)

output_lines = []
for word in words_to_embed:
    vec_p = p_dict[word]
    vec_b = bert_dict[word]
    combined_vector = np.concatenate([vec_p, vec_b])

    vector_str = " ".join([str(v) for v in combined_vector])
    output_lines.append(f"{word} {vector_str}")

filename = 'word_embedings_file.txt'
with open(filename, 'w', encoding='utf-8') as f:
    f.write('\n'.join(output_lines))

Some weights of the model checkpoint at allegro/herbert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.sso.sso_relationship.bias', 'cls.sso.sso_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
